https://colab.research.google.com/github/NadirEM/nlp-notebooks/blob/master/Fine_tune_ALBERT_sentence_pair_classification.ipynb#section04

In [3]:
!pip install datasets
!pip install transformers

     |████████████████████████████████| 163kB 10.0MB/s 
     |████████████████████████████████| 17.7MB 204kB/s 
     |████████████████████████████████| 245kB 53.8MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1
     |████████████████████████████████| 1.3MB 8.7MB/s 
     |████████████████████████████████| 1.1MB 23.6MB/s 
     |████████████████████████████████| 2.9MB 44.2MB/s 
     |████████████████████████████████| 890kB 41.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=14f03b7f18c9f14cb348495d8dba21f3a902acbb64365cac859c02d911581f43
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
import torch
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import copy
import torch.optim as optim
import random
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
from datasets import load_dataset, load_metric


In [5]:
# Load the MRPC dataset (train, validation and test)
# GLUE(Ganeral Language Understanding Evaluation) : 말 그대로 다양한 nlu task의 집합체다. GLUE의 대부분의 dataset은 이미 존재하던것이었지만, train, dev, test splits으로 나눠진다.(squad데이터와 비슷한 것)
# MRPC : GLUE 안의 데이터 종류, MRPC(Microsoft Research Paraphrate Corpus): 문장쌍의 유사성 확인하는 문제.
dataset = load_dataset('glue', 'mrpc')

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4. Subsequent calls will reuse this data.


In [6]:
split = dataset['train'].train_test_split(test_size=0.1, seed=1) # train데이터 분리 
# print('split : ', len(split)) # train, test 총 2개
# print(split) # split : features: ['sentence1', 'sentence2', 'label', 'idx'] 
train = split['train'] # train의 90% 훈련데이터로
# print('train : ', len(train))
# print(train['sentence1'][:1])
# print(train['sentence2'][:1])
val = split['test'] # train의 10% 검증데이터로
# print('val : ', len(val))
test = dataset['validation'] # 데이터 세트 라이브러리에서 테스트 레이블을 사용할 수 없기 때문에 원래 유효성 검사 데이터가 테스트 데이터로 사용됩니다.
# print('test : ', len(test))

df_train = pd.DataFrame(train)
df_val = pd.DataFrame(val)
df_test = pd.DataFrame(test)

In [7]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(3301, 4)
(367, 4)
(408, 4)


In [8]:
df_train.head()

,idx,label,sentence1,sentence2
0,3349,1,"The American troops , who also defend the mayo...","The American troops from 3-15 infantry , who a..."
1,1806,0,"Last week , his lawyers asked Warner to grant ...","Last week , his lawyers asked Gov. Mark R. War..."
2,2681,1,Their election increases the board from seven ...,Their appointments increase Berkshire 's board...
3,2001,1,"Dolores Mahoy , 68 , of Colorado Springs , Col...","Dolores E. Mahoy , 68 , of Colorado Springs is..."
4,2993,1,""" They were an inspirational couple , selfless...",He said : “ They were an inspirational couple ...


In [9]:
# 문장을 토큰화 처리하는 함수(토큰화, 패딩, 문장 구분)
class CustomDataset(Dataset):
    def __init__(self, data, maxlen, with_labels=True, bert_model='albert-base-v2'): # albert-base-v2 모델 사용
        self.data = data
        self.tokenizer = AutoTokenizer.from_pretrained(bert_model)
        
        self.maxlen = maxlen
        self.with_labels = with_labels
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
      # 데이터 프레임의 지정된 인덱스에서 문장 1과 문장 2 선택
        sent1 = str(self.data.loc[index, 'sentence1'])
        sent2 = str(self.data.loc[index, 'sentence2'])
        
        # 토큰 ID, attention mask 및 token type ids를 얻기 위해 문장 쌍을 토큰 화합니다.
        # truncation : 잘림표시없이 표시 / ...처럼 나오는 경우
        encoded_pair = self.tokenizer(sent1, sent2, padding='max_length', truncation=True, max_length=self.maxlen, return_tensors='pt') # albert-base-v2 모델로 tokenizer실행
        # print(encoded_pair) # label이 input_ids / attention_mask / token_type_ids가 있음
        token_ids = encoded_pair['input_ids'].squeeze(0) # 문장 토큰화
        attn_masks = encoded_pair['attention_mask'].squeeze(0) # 패딩 된 값은 "0"이고 다른 값은 "1"(패딩 역할)
        token_type_ids = encoded_pair['token_type_ids'].squeeze(0) # 첫 번째 문장 토큰의 경우 "0"이고 두 번째 문장 토큰의 경우 "1"인 이진 텐서(문장 구분)
        
        if self.with_labels: # 데이터에 라벨이 존재할때
            label = self.data.loc[index, 'label']
            return token_ids, attn_masks, token_type_ids, label
        else:
            return token_ids, attn_masks, token_type_ids

In [10]:
# 토큰화, 패딩, 구분된 것들을 bert모델에 적용시켜 하나의 tensor로 표시
class SentencePairClassifier(nn.Module):

    def __init__(self, bert_model="albert-base-v2", freeze_bert=False):
        super(SentencePairClassifier, self).__init__()
        self.bert_layer = AutoModel.from_pretrained(bert_model)
        
        # bert-model의 종류에 따른 hidden_state size결정
        if bert_model == "albert-base-v2":  # 12M parameters
            hidden_size = 768
        elif bert_model == "albert-large-v2":  # 18M parameters
            hidden_size = 1024
        elif bert_model == "albert-xlarge-v2":  # 60M parameters
            hidden_size = 2048
        elif bert_model == "albert-xxlarge-v2":  # 235M parameters
            hidden_size = 4096
        elif bert_model == "bert-base-uncased": # 110M parameters
            hidden_size = 768

        # bert-layer를 고정시키고 분류 레이어 가중치만 훈련
        if freeze_bert:
            for p in self.bert_layer.parameters():
                p.requires_grad = False

        self.cls_layer = nn.Linear(hidden_size, 1)

        self.dropout = nn.Dropout(p=0.1)

    @autocast() # autocast실행(혼합 정밀도로 실행) 
    def forward(self, input_ids, attn_masks, token_type_ids):
        '''
        입력 :
            -input_ids : token ids를 포함하는 텐서(문장 토큰화)
            -attn_masks : 패딩되지 않은 값에 초점을 맞추는 데 사용할주의 mask를 포함하는 텐서(패딩 처리)
            -token_type_ids : 문장 1과 문장 2를 식별하는 데 사용할 토큰 유형 ID를 포함하는 텐서(문장 구분)
        '''
        # 문맥화 된 표현을 얻기 위해 BERT 기반 모델에 입력을 공급
        # 토큰화, 패딩, 구분된 것을 입력으로 넣어 bert모델으로 적용시킴
        cont_reps, pooler_output = self.bert_layer(input_ids, attn_masks, token_type_ids)
        # print('cont_reps : ',cont_reps)
        # print('pooler_output : ',pooler_output)

        # 분류 자 ​​레이어에 [CLS] 토큰의 숨겨진 상태를 마지막 레이어에 공급합니다.
        # 선형 레이어 및 Tanh 활성화. 선형 레이어 가중치는 문장 순서 예측 (ALBERT) 또는 다음 문장 예측 (BERT)에서 학습되었습니다.
        # 사전 훈련 중 목표.
        logits = self.cls_layer(self.dropout(pooler_output))

        return logits

In [11]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHEED'] = str(seed)
    
def evaluate_loss(net, device, criterion, dataloader):
    net.eval()
    
    mean_loss = 0
    count = 0
    
    with torch.no_grad():
        for it, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm(dataloader)):
            seq, attn_masks, token_type_ids, labels = seq.to(device), attn_masks.to(device), token_type_ids.to(device), labels.to(device)
            logits = net(seq, attn_masks, token_type_ids)
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            count += 1
            
    return mean_loss / count

In [12]:
def train_bert(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate):

    best_loss = np.Inf
    best_ep = 1
    nb_iterations = len(train_loader)
    print_every = nb_iterations // 5  # print the training loss 5 times per epoch
    iters = []
    train_losses = []
    val_losses = []

    scaler = GradScaler()

    for ep in range(epochs):

        net.train()
        running_loss = 0.0
        for it, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm(train_loader)):

            # Converting to cuda tensors
            seq, attn_masks, token_type_ids, labels = \
                seq.to(device), attn_masks.to(device), token_type_ids.to(device), labels.to(device)
    
            # Enables autocasting for the forward pass (model + loss)
            with autocast():
                # Obtaining the logits from the model
                logits = net(seq, attn_masks, token_type_ids)

                # Computing loss
                loss = criterion(logits.squeeze(-1), labels.float())
                loss = loss / iters_to_accumulate  # Normalize the loss because it is averaged

            # 손실을 조정합니다. 스케일링 된 손실에 대해 backward ()을 호출하여 스케일링 된 그래디언트를 만듭니다.
            scaler.scale(loss).backward()

            if (it + 1) % iters_to_accumulate == 0:
                scaler.step(opti)
                scaler.update()
                lr_scheduler.step()
                opti.zero_grad()

            running_loss += loss.item()

            if (it + 1) % print_every == 0:  # Print training loss information
                print()
                print("Iteration {}/{} of epoch {} complete. Loss : {} "
                      .format(it+1, nb_iterations, ep+1, running_loss / print_every))

                running_loss = 0.0

        val_loss = evaluate_loss(net, device, criterion, val_loader)  # Compute validation loss
        print()
        print("Epoch {} complete! Validation Loss : {}".format(ep+1, val_loss))

        if val_loss < best_loss:
            print("Best validation loss improved from {} to {}".format(best_loss, val_loss))
            print()
            net_copy = copy.deepcopy(net)  # save a copy of the model
            best_loss = val_loss
            best_ep = ep + 1

    # Saving the model
    path_to_model='sentence_pair_classification.pt'
    torch.save(net_copy.state_dict(), path_to_model)
    print("The model has been saved in {}".format(path_to_model))

    del loss
    torch.cuda.empty_cache()

In [13]:
bert_model = "albert-base-v2"  
freeze_bert = False  
maxlen = 128  
bs = 16  
iters_to_accumulate = 2  
lr = 2e-5  
epochs = 4  

In [14]:
#  Set all seeds to make reproducible results
set_seed(1)

# Creating instances of training and validation set
print("Reading training data...")
train_set = CustomDataset(df_train, maxlen, bert_model)
print("Reading validation data...")
val_set = CustomDataset(df_val, maxlen, bert_model)
# Creating instances of training and validation dataloaders
train_loader = DataLoader(train_set, batch_size=bs, num_workers=5)
val_loader = DataLoader(val_set, batch_size=bs, num_workers=5)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = SentencePairClassifier(bert_model, freeze_bert=freeze_bert)

net.to(device)

criterion = nn.BCEWithLogitsLoss()
opti = AdamW(net.parameters(), lr=lr, weight_decay=1e-2)
num_warmup_steps = 0 # The number of steps for the warmup phase.
num_training_steps = epochs * len(train_loader)  # The total number of training steps
t_total = (len(train_loader) // iters_to_accumulate) * epochs  # Necessary to take into account Gradient accumulation
lr_scheduler = get_linear_schedule_with_warmup(optimizer=opti, num_warmup_steps=num_warmup_steps, num_training_steps=t_total)

train_bert(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate)

Reading training data...



Reading validation data...


 20%|██        | 42/207 [00:07<00:27,  6.04it/s]


Iteration 41/207 of epoch 1 complete. Loss : 0.3030669918147529 


 40%|████      | 83/207 [00:14<00:20,  6.17it/s]

...


Iteration 164/207 of epoch 4 complete. Loss : 0.08075261097855685 


100%|█████████▉| 206/207 [00:35<00:00,  5.88it/s]


Iteration 205/207 of epoch 4 complete. Loss : 0.0734972147861632 


100%|██████████| 23/23 [00:01<00:00, 11.82it/s]



Epoch 4 complete! Validation Loss : 0.33957109373548755
Best validation loss improved from 0.3486955897963565 to 0.33957109373548755

The model has been saved in sentence_pair_classification.pt


In [15]:
def get_probs_from_logits(logits):
    """
    시그모이드 함수를 적용하여 로직의 텐서를 확률 배열로 변환
    """
    probs = torch.sigmoid(logits.unsqueeze(-1))
    return probs.detach().cpu().numpy()

def test_prediction(net, device, dataloader, with_labels=True, result_file="output.txt"):
    """
    레이블이 있거나 없는 데이터 세트의 확률을 예측하고 결과를 파일로 인쇄합니다.
    """
    net.eval()
    w = open(result_file, 'w')
    probs_all = []

    with torch.no_grad():
        if with_labels:
            for seq, attn_masks, token_type_ids, _ in tqdm(dataloader):
                seq, attn_masks, token_type_ids = seq.to(device), attn_masks.to(device), token_type_ids.to(device)
                logits = net(seq, attn_masks, token_type_ids)
                probs = get_probs_from_logits(logits.squeeze(-1)).squeeze(-1)
                probs_all += probs.tolist()
        else:
            for seq, attn_masks, token_type_ids in tqdm(dataloader):
                seq, attn_masks, token_type_ids = seq.to(device), attn_masks.to(device), token_type_ids.to(device)
                logits = net(seq, attn_masks, token_type_ids)
                probs = get_probs_from_logits(logits.squeeze(-1)).squeeze(-1)
                probs_all += probs.tolist()

    w.writelines(str(prob)+'\n' for prob in probs_all)
    w.close()

In [16]:
path_to_model = 'sentence_pair_classification.pt'  
# path_to_model = '/content/models/...'  # You can add here your trained model

path_to_output_file = 'output.txt'

print("Reading test data...")
test_set = CustomDataset(df_test, maxlen, bert_model)
test_loader = DataLoader(test_set, batch_size=bs, num_workers=5)

model = SentencePairClassifier(bert_model)
if torch.cuda.device_count() > 1:  # if multiple GPUs
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

print()
print("Loading the weights of the model...")
model.load_state_dict(torch.load(path_to_model))
model.to(device)

print("Predicting on test data...")
test_prediction(net=model, device=device, dataloader=test_loader, with_labels=True,  # set the with_labels parameter to False if your want to get predictions on a dataset without labels
                result_file=path_to_output_file)
print()
print("Predictions are available in : {}".format(path_to_output_file))

Reading test data...


  0%|          | 0/26 [00:00<?, ?it/s]


Loading the weights of the model...
Predicting on test data...


100%|██████████| 26/26 [00:02<00:00, 11.78it/s]


Predictions are available in : output.txt


In [17]:
path_to_output_file = 'output.txt'  # path to the file with prediction probabilities

labels_test = df_test['label']  # true labels

probs_test = pd.read_csv(path_to_output_file, header=None)[0]  # prediction probabilities
threshold = 0.5   # you can adjust this threshold for your own dataset
preds_test=(probs_test>=threshold).astype('uint8') # predicted labels using the above fixed threshold

metric = load_metric("glue", "mrpc")

In [18]:
metric._compute(predictions=preds_test, references=labels_test)

{'accuracy': 0.875, 'f1': 0.9100529100529101}